In [17]:
import hyperspy.api as hs
from skimage.feature import peak_local_max
import numpy as np
from scipy.optimize import curve_fit

def two_dim_gaussian(x_data_tuple, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    (x, y) = x_data_tuple
    xo = float(xo)
    yo = float(yo)
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    return offset + amplitude*np.exp(- (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) + c*((y-yo)**2)))

def fit_gaussian_peak(image_data, x, y, window_size=5):
    x = int(x)
    y = int(y)
    data = image_data[y-window_size:y+window_size+1, x-window_size:x+window_size+1]

    x_data, y_data = np.meshgrid(np.arange(data.shape[1]), np.arange(data.shape[0]))
    x_data_tuple = (x_data.ravel(), y_data.ravel())
    data = data.ravel()

    initial_guess = (data.max(), window_size, window_size, 3, 3, 0, data.min())
    popt, _ = curve_fit(two_dim_gaussian, x_data_tuple, data, p0=initial_guess)

    return x - window_size + popt[1], y - window_size + popt[2]

import pymc3 as pm

def fit_gaussian_peak_bayesian(image_data, x, y, window_size=20):
    x = int(x)
    y = int(y)
    data = image_data[y-window_size:y+window_size+1, x-window_size:x+window_size+1]

    x_data, y_data = np.meshgrid(np.arange(data.shape[1]), np.arange(data.shape[0]))

    with pm.Model() as model:
        amplitude = pm.Uniform('amplitude', lower=0, upper=data.max()*2)
        xo = pm.Uniform('xo', lower=0, upper=data.shape[1])
        yo = pm.Uniform('yo', lower=0, upper=data.shape[0])
        sigma_x = pm.Uniform('sigma_x', lower=0, upper=10)
        sigma_y = pm.Uniform('sigma_y', lower=0, upper=10)
        theta = pm.Uniform('theta', lower=-np.pi, upper=np.pi)
        offset = pm.Uniform('offset', lower=0, upper=data.max())

        def two_dim_gaussian_wrap(x, y, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
            return two_dim_gaussian((x, y), amplitude, xo, yo, sigma_x, sigma_y, theta, offset)

        gauss = pm.Deterministic('gauss', two_dim_gaussian_wrap(x_data, y_data, amplitude, xo, yo, sigma_x, sigma_y, theta, offset))

        obs = pm.Normal('obs', mu=gauss, sd=1, observed=data)

        trace = pm.sample(1000, tune=1000, chains=2, cores=1, return_inferencedata=False)

    summary = pm.summary(trace)
    return x - window_size + summary.loc['xo', 'mean'], y - window_size + summary.loc['yo', 'mean']

# ファイル名を指定します。
file_name = 'SrTiO3_50cell_simu_intensity50.dm4'

# dm3ファイルを読み込みます。
image = hs.load(file_name)
data = image.data

# 輝点の座標を見つけます。
min_distance = 10
threshold_abs = 20
coordinates = peak_local_max(data, min_distance=min_distance, threshold_abs=threshold_abs)

# 輝点を2次元ガウス関数でフィッティングし、サブピクセル精度で座標を計算します。
refined_coordinates = []
for coord in coordinates:
    x, y = fit_gaussian_peak(data, coord[1], coord[0])
    refined_coordinates.append((y, x))

# 輝点の座標を出力します。
print('輝点のサブピクセル精度での座標:')
for coord in refined_coordinates:
    print(coord)

輝点のサブピクセル精度での座標:
(39.107558923073604, 429.1808869361873)
(1248.3079917392015, 1833.5601891640192)
(1482.182319406609, 546.2301518715931)
(1053.1305809717242, 1443.3937191326854)
(936.3250804022967, 194.85468247752857)
(780.1248044858139, 1599.2008173010172)
(390.3948956208352, 1482.4009529765149)
(273.0435615695107, 1443.146487935878)
(975.257414044646, 1170.200261540755)
(117.09407075951133, 1287.281880441582)
(1209.055695759437, 819.1536002511727)
(1326.3300609126086, 1599.2974552281594)
(858.3659815566465, 1794.3790322729783)
(1560.649223544803, 351.2165714985662)
(1404.3692186525109, 1053.0972500257321)
(195.0944735010965, 1092.272868912536)
(663.0903407029988, 624.3633014207604)
(624.1869368548101, 273.0224725729452)
(819.2188124522806, 156.02381437005735)
(156.21321823300826, 195.14825454595214)
(936.3432156127496, 272.90548838155536)
(78.06907134570487, 1209.18635115488)
(1209.4764104477326, 1482.4602818327792)
(1677.3094479737763, 1833.7518247087237)
(1326.5784942310395, 897.02